In [ ]:
from dataclasses import dataclass as dc


In [ ]:
@dc
class Tarefa:
  numero: int
  tempo_q_demora: int
  acabar_antes: int = None

  @property
  def comecar_antes(self):
    return self.acabar_antes - self.tempo_q_demora
  @comecar_antes.setter
  def comecar_antes(self, value):
    self.acabar_antes = value + self.tempo_q_demora

In [ ]:
@dc
class Maquina:
  numero: int
  n_tarefas_maximo: int

  def __post_init__(self):
    self.tarefas = []
  def queue(self, tarefa):
    self.tarefas.append(tarefa)
  def reset(self):
    self.tarefas = []
  @property
  def horario(self):
    horario = [] # o horario representa quando muda de tarefa para a proxima
    tempo = 0
    for tarefa in self.tarefas:
      horario.append(tempo)
      tempo += tarefa.tempo_q_demora
    return horario
  def get_tarefa_at(self, hora):
    tempo = 0
    for tarefa in self.tarefas:
      tempo += tarefa.tempo_q_demora
      if tempo > hora:
        return tarefa
    return None

In [ ]:
M1 = Maquina(1, 4)
M2 = Maquina(2, 3)
M3 = Maquina(3, 3)
Maquinas = [M1, M2, M3]

In [ ]:
Tarefas = [
  Tarefa(
    numero=i+1,
    tempo_q_demora=t
  )
  for i, t in enumerate([38,33,36,20,32,29,46,38,34,40])
]
def getTarefa(i):
  return Tarefas[i-1]
  
# 1 junho (8h) até 5 junho (8h)
[getTarefa(i).acabar_antes = 24*(5-1) for i in [3,4,5]]